In [1]:

from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Import NasNet and MobileNet

In [0]:
import tensorflow as tf

def MobileNet(img_size = (224, 224, 3)):
    
    mobilenet = tf.keras.applications.MobileNetV2(
                                                   input_shape=img_size,
                                                   include_top=False,
                                                   weights='imagenet'
                                                )
    
    for layer in mobilenet.layers[:-5]:
        layer.trainable = False
        
    return mobilenet
  
def NasNet(img_size = (224, 224, 3)):
    
    nasnet = tf.keras.applications.nasnet.NASNetMobile(
                                                   input_shape=img_size,
                                                   include_top=False,
                                                   weights='imagenet'
                                                )
    
    for layer in nasnet.layers[:-15]:
        layer.trainable = False
        
    return nasnet

Prepare preprocessing code

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2

label_dict = {
        "akiec":0,
        "bcc":  1,
        "bkl":  2,
        "df":   3,
        "mel":  4,
        "nv":   5,
        "vasc": 6
        }

def load_data(img_parent_path, meta_path, img_shape):
    
    metadata_df = pd.read_csv(meta_path)
    
    images = []
    labels = []
    
    for index, row in metadata_df.iterrows():
        image_id = row['image_id']
        
        label_abbv = row['dx']
        label = label_dict[label_abbv]
        
        # one-hot encode label
        temp = np.zeros((7))
        temp[label] = 1
        label = temp
        
        image_path = img_parent_path + image_id + ".jpg"
        
        image = np.asarray(cv2.resize(cv2.imread(image_path), (224, 224)), dtype=int)
        
        # we have to reverse the RGB order as cv2 uses BGR
        image = image[:,:,::-1]
        
        #normalize data
        image = image / 256.
        
        images.append(image)
        labels.append(label)

        if index % 10 == 0:
            print("Loading Image", index)

        if index > 2500:
            break
        
    return np.asarray(images), np.asarray(labels)

Test models

In [4]:
nasnet = NasNet()
mobilenet = MobileNet()

# check layers for training
print("NASNET FINAL 15 LAYERS: ")
for layer in nasnet.layers[-50:]:
  print(layer)
print("MOBILENET FINAL 15 LAYERS: ")
for layer in mobilenet.layers[-15:]:
  print(layer)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
NASNET FINAL 15 LAYERS: 
MOBILENET FINAL 15 LAYERS: 


Create data generator to avoid loading all images into RAM at once

In [5]:
from sklearn.model_selection import train_test_split as tts

METADATA_PATH = "/content/drive/My Drive/HTN2019/HAM10000_metadata.csv"
IMG_PARENT_PATH = "/content/drive/My Drive/HTN2019/HAM10000_images_part_1/"

print("NASNET FINAL LAYERS: ", nasnet.layers[-10:])
print("MOBILENET FINAL LAYERS: ", mobilenet.layers[-10:])

# first we will try training just the classification layer

images, labels = load_data(IMG_PARENT_PATH, METADATA_PATH, (224, 224, 3))
x_train, x_test, y_train, y_test = tts(images, labels, test_size=0.25)

# checkpoint callbacks
nas_path = "/content/drive/My Drive/HTN2019/Checkpoints/NasNet/3e_10k_5l"
mobile_path = "/content/drive/My Drive/HTN2019/Checkpoints/MobileNet/3e_10k_5l"

nas_callback = tf.keras.callbacks.ModelCheckpoint(
    nas_path, verbose=1, save_weights_only=False,
    save_freq='epoch')

mobile_callback = tf.keras.callbacks.ModelCheckpoint(
    mobile_path, verbose=1, save_weights_only=False,
    save_freq='epoch')


# implement classification head for both nets

nasnet_full = tf.keras.Sequential([
        nasnet, 
        tf.keras.layers.Flatten(),
        # dropout to avoid overfitting
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(7, activation='softmax'),
        ])

mobilenet_full = tf.keras.Sequential([
        mobilenet,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(7, activation='softmax')
        ])


print("COMPILING NASNET")
nasnet_full.compile(optimizer=tf.keras.optimizers.RMSprop(),
                    loss='categorical_crossentropy',
                    metrics=['accuracy']
                    )

print("TRAINING NASNET")
nasnet_full.fit(x_train, y_train, epochs=2, callbacks=[nas_callback])


print("COMPILING MOBILENET")
mobilenet_full.compile(optimizer=tf.keras.optimizers.RMSprop(),
                    loss='categorical_crossentropy',
                    metrics=['accuracy']
                    )

print("TRAINING MOBILENET")
mobilenet_full.fit(x_train, y_train, epochs=2, callbacks=[mobile_callback])

nas_score = nasnet_full.evaluate(x_test, y_test)
mobile_score = mobilenet_full.evaluate(x_test, y_test)

print("NASNET LOSS: {} - NASNET ACCURACY: {}".format(nas_score[0], nas_score[1]))
print("MOBILENET LOSS: {} - MOBILENET ACCURACY: {}".format(mobile_score[0], mobile_score[1]))

NASNET FINAL LAYERS:  [<tensorflow.python.keras.layers.pooling.AveragePooling2D object at 0x7fda4dac0908>, <tensorflow.python.keras.layers.pooling.AveragePooling2D object at 0x7fda4daeaf60>, <tensorflow.python.keras.layers.normalization.BatchNormalization object at 0x7fda4d98b550>, <tensorflow.python.keras.layers.merge.Add object at 0x7fda4e2e0710>, <tensorflow.python.keras.layers.merge.Add object at 0x7fda4e2c95f8>, <tensorflow.python.keras.layers.merge.Add object at 0x7fda4dac0cc0>, <tensorflow.python.keras.layers.merge.Add object at 0x7fda4daf1518>, <tensorflow.python.keras.layers.merge.Add object at 0x7fda4daf1908>, <tensorflow.python.keras.layers.merge.Concatenate object at 0x7fda4e2c9390>, <tensorflow.python.keras.layers.core.Activation object at 0x7fda4fb88ba8>]
MOBILENET FINAL LAYERS:  [<tensorflow.python.keras.layers.normalization.BatchNormalization object at 0x7fda4483e3c8>, <tensorflow.python.keras.layers.advanced_activations.ReLU object at 0x7fda4483ecc0>, <tensorflow.pytho

OSError: ignored